In [1]:
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video

prompt = "A panda, dressed in a small, red jacket and a tiny hat, sits on a wooden stool in a serene bamboo forest. The panda's fluffy paws strum a miniature acoustic guitar, producing soft, melodic tunes. Nearby, a few other pandas gather, watching curiously and some clapping in rhythm. Sunlight filters through the tall bamboo, casting a gentle glow on the scene. The panda's face is expressive, showing concentration and joy as it plays. The background includes a small, flowing stream and vibrant green foliage, enhancing the peaceful and magical atmosphere of this unique musical performance."
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    torch_dtype=torch.bfloat16
)

pipe.enable_model_cpu_offload()
pipe.vae.enable_tiling()

video = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

export_to_video(video, "output.mp4", fps=8)


/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

AttributeError: Found an existing imageio backend in your environment. Attempting to export video with imageio. 
Unable to find a compatible ffmpeg installation in your environment to use with imageio. Please install via `pip install imageio-ffmpeg

In [1]:
from datasets import load_dataset
# Replace 'your_dataset_name' with the actual dataset name
dataset = load_dataset("yesidobyte/nsfw1024", split='train')
# Specify the directory to save images
save_dir = '/workspace/data/samples_nudity/images'
import os
import requests
from PIL import Image
from io import BytesIO
# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
# Select the first 200 images
first_200_images = dataset.select(range(200))
# Access the images
for i, example in enumerate(first_200_images):
    # Replace 'image_column' with the actual column name containing image URLs or data
    image = example['image']
    try:
        # If the image is a URL, download it
        if isinstance(image, str):
            response = requests.get(image)
            with open(f"{save_dir}/image_{i}.jpg", 'wb') as f:
                f.write(response.content)
        # If the image is a PIL Image object
        elif isinstance(image, Image.Image):
            with open(f"{save_dir}/image_{i}.jpg", 'wb') as f:
                image.save(f, format='JPEG')
    except:
        pass

Generating train split:   0%|          | 0/4289 [00:00<?, ? examples/s]

In [7]:
with open('/workspace/data/samples_nudity/caption.txt', 'w') as file:
    for _ in range(200):
        file.write('naked woman\n')

In [ ]:
!cd /workspace/concept-ablation/diffusers  && accelerate launch train.py \
          --pretrained_model_name_or_path="THUDM/CogVideoX-5b"  \
          --output_dir="/workspace/log_ouput" \
          --class_data_dir="/workspace/data/samples_nudity" \
          --class_prompt="people, body"  \
          --caption_target "nudity, nsfw" \
          --concept_type nudity \
          --resolution=512  \
          --train_batch_size=2  \
          --learning_rate=2e-7  \
          --max_train_steps=4 \
          --scale_lr --hflip --noaug \
          --train_size 100 \
          --lr_warmup_steps 10 \
          --use_8bit_adam \
          --parameter_group full-weight \
          --enable_xformers_memory_efficient_attention

[2024-09-05 04:46:03,876] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
W0905 04:46:04.990000 140422967673984 torch/distributed/run.py:779] 
W0905 04:46:04.990000 140422967673984 torch/distributed/run.py:779] *****************************************
W0905 04:46:04.990000 140422967673984 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0905 04:46:04.990000 140422967673984 torch/distributed/run.py:779] *****************************************
/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::